In [2]:
import numpy as np
from transformers import pipeline, set_seed
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
import re

# Read the file into a list of lines
with open('Group_messages.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Initialize lists to store the parsed data
timestamps = []
users = []
messages = []

# Regular expression pattern for matching the timestamp and username
pattern = r'^\[(.*?)\] (.*)$'

# Temporary variables to hold the current user and message
current_user = None
current_message = ""

# Iterate over the lines and parse them
for line in lines:
    line = line.strip()
    
    if line == "":  # Skip empty lines
        continue
    
    match = re.match(pattern, line)
    
    if match:  # If the line starts with a timestamp and username
        if current_user is not None:
            timestamps.append(current_timestamp)
            users.append(current_user)
            messages.append(current_message.strip())
        
        current_timestamp = match.group(1)
        current_user = match.group(2)
        current_message = ""
    else:
        current_message += line + " "  # Continue appending to the current message
    
# Append the last message
if current_user is not None:
    timestamps.append(current_timestamp)
    users.append(current_user)
    messages.append(current_message.strip())
# Create a DataFrame from the parsed data
df = pd.DataFrame({
    'Timestamp': timestamps,
    'User': users,
    'Message': messages
})

# Display the DataFrame
print(df.head())
dataset = load_dataset('text', data_files={'train': 'your_data.txt'})
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
)

# Train the model
trainer.train()

generator = pipeline('text-generation', model='./gpt2-finetuned')
generator("Your input text here", max_length=30, num_return_sequences=5)


FileNotFoundError: Unable to find 'D:/project/yoakims-chatbot-trolling\your_data.txt'